Overall feedback: More context on what we're looking for/measuring here would be helpful

In [0]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt 
import numpy as np
import statsmodels.api as sm
# You can use pd.set_option() to make sure you see all the rows and columns in your dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Step 1

In [3]:
df = pd.read_csv('/content/drive/My Drive/Data for Python/cleaned_311_data_revised.csv', header='infer')
homeless = pd.read_csv('https://data.cityofnewyork.us/api/views/3qem-6v3v/rows.csv?accessType=DOWNLOAD', header='infer')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,19,20,22,33,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Step 2

In [29]:
homeless_093019 = homeless[homeless['Report Date']== '09/30/2019']
homeless_093019.columns

Index(['Report Date', 'Borough', 'Community District',
       'Adult Family Comm Hotel', 'Adult Family Shelter', 'Adult Shelter',
       'Adult Shelter Comm Hotel', 'FWC Cluster', 'FWC Comm Hotel',
       'FWC Shelter'],
      dtype='object')

Step 3

In [0]:
col_list = ['Adult Family Comm Hotel', 'Adult Family Shelter', 'Adult Shelter', 'Adult Shelter Comm Hotel', 'FWC Cluster', 'FWC Comm Hotel', 'FWC Shelter']
homeless_093019['shelter_count'] = homeless_093019[col_list].sum(axis=1)
#different from solution...fillna default with sum and not needed? Axis may confuse people though. This took me a minute. More help/hint needed
homeless_093019

Step 4/5

In [0]:
homeless_311 = df[df['complaint_type'] == 'Homeless Person Assistance']
homeless_311_borocd_count = homeless_311.groupby('borocd').size().reset_index(name='count_homeless_311_requests')

Step 6

In [44]:
homeless_data = pd.merge(left=homeless_311_borocd_count, right=homeless_093019, left_on='borocd', right_on='Community District')
#seemed to work ok w/o converting to string or number?
homeless_data.head(5)

,borocd,count_homeless_311_requests,Report Date,Borough,Community District,Adult Family Comm Hotel,Adult Family Shelter,Adult Shelter,Adult Shelter Comm Hotel,FWC Cluster,FWC Comm Hotel,FWC Shelter,shelter_count
0,101.0,758,09/30/2019,Manhattan,101,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
1,103.0,520,09/30/2019,Manhattan,103,NaN,NaN,7.0,2.0,3.0,2.0,3.0,17.0
2,104.0,1055,09/30/2019,Manhattan,104,NaN,1.0,3.0,NaN,NaN,4.0,NaN,8.0
3,105.0,3020,09/30/2019,Manhattan,105,NaN,1.0,NaN,NaN,NaN,8.0,1.0,10.0
4,106.0,753,09/30/2019,Manhattan,106,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0


This Should be Step 7? 

In [0]:
plt.figure(figsize=(15, 6))

plt.scatter(
    np.array(homeless_data.shelter_count), # x
    np.array(homeless_data.count_homeless_311_requests), # y
    s=50, # size of marker
    c='#3d03fc', # color hex color picker: https://www.google.com/search?q=color+picker
    edgecolors='black', # outline color of the marker
    alpha=0.5) # transparency. 0 is 100% transparent. 1 is 100% opaque
plt.title('Shelter Count vs 311 Homeless Assistance Requests by Community District')
plt.xlabel('Shelter Count')
plt.ylabel('# of 311 Requests in Community District')
    
plt.show()

#Ask if it look likes there's a linear relationship? 

This shoudl be step 8

In [69]:
np.corrcoef(homeless_data.shelter_count, homeless_data.count_homeless_311_requests)
#sk if the correlation is high or low

array([[ 1.        , -0.00781267],
       [-0.00781267,  1.        ]])

Dunno whether to recommend they do a linear regression or not. I'm concerned the data doesn't seem to satisfy linear regression assumptions so my guess is to say that they should proceed but I don't know enough to test this/back this up. It's already a lot of work? 
